In [ ]:
# --- Librerías Fundamentales ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# --- Preprocesamiento y Modelado ---
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    r2_score, mean_squared_error, roc_auc_score, roc_curve
)

# --- Modelos Adicionales (Opcional pero recomendado) ---
try:
    from xgboost import XGBClassifier, XGBRegressor
    xgb_available = True
except ImportError:
    xgb_available = False
    print("XGBoost no está instalado. Algunas funcionalidades avanzadas de modelado no estarán disponibles.")

# --- Librerías Geográficas y Visualización Interactiva ---
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import plotly.express as px

# --- Configuraciones Adicionales ---
%matplotlib inline 
sns.set_style("whitegrid") # Estilo de gráficos más limpio
plt.rcParams['figure.figsize'] = (10, 6) # Tamaño de figura por defecto
warnings.filterwarnings('ignore', category=FutureWarning) # Ignorar warnings futuros de librerías
warnings.filterwarnings('ignore', category=UserWarning) # Ignorar warnings de usuario (e.g., de Seaborn)

print("Librerías cargadas exitosamente.")
if xgb_available:
    print("XGBoost está disponible ✅")

### Celda 1: Importación de Librerías y Configuración Inicial

**Propósito:** Cargar todas las herramientas necesarias para el análisis de datos, visualización y machine learning.

**Detalles del Código:**
*   **Librerías Fundamentales:** `pandas`, `numpy`, `matplotlib.pyplot`, `seaborn`, `warnings`.
*   **Preprocesamiento y Modelado (`sklearn`):** Herramientas para división de datos, optimización, preprocesamiento (escalado, codificación, imputación), pipelines, modelos (Regresión Logística, Random Forest) y métricas.
*   **Modelos Adicionales:** Intenta importar `XGBoost` y establece una bandera `xgb_available`.
*   **Librerías Geográficas y Visualización Interactiva:** `geopandas`, `folium`, `plotly.express`.
*   **Configuraciones Adicionales:** `%matplotlib inline` para mostrar gráficos en Jupyter, estilo `whitegrid` para `seaborn`, tamaño de figura por defecto, y filtros de warnings.

**Resultado Esperado:**
Mensajes indicando que las librerías se cargaron y si XGBoost está disponible.

In [ ]:
# Cargar el dataset desde URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/_0eYOqji3unP1tDNKWZMjg/weatherAUS-2.csv"
df_original = pd.read_csv(url)

print("Dataset original cargado:")
df_original.head()

### Celda 2: Carga del Conjunto de Datos

**Propósito:** Obtener los datos climáticos desde la URL proporcionada.

**Detalles del Código:**
*   Se define la `url` donde se encuentra el archivo CSV.
*   `pd.read_csv(url)` lee el archivo CSV y lo carga en un DataFrame de pandas llamado `df_original`.
*   `df_original.head()` muestra las primeras 5 filas del DataFrame para una inspección inicial.

**Resultado Esperado:**
Una tabla con las primeras cinco filas del conjunto de datos `weatherAUS`.

In [ ]:
# Información general del dataset completo
print("--- Información General del Dataset Completo ---")
df_original.info()
print(f"\nDimensiones del dataset original: {df_original.shape}")

# --- Paso 1: Filtrar por ubicaciones de interés ---
# Mantendremos el enfoque en Melbourne y sus alrededores como en el notebook original.
locations_of_interest = ['Melbourne', 'MelbourneAirport', 'Watsonia']
df = df_original[df_original['Location'].isin(locations_of_interest)].copy() # Usar .copy() para evitar SettingWithCopyWarning
print(f"\nDataset filtrado por ubicaciones {locations_of_interest}.")
print(f"Dimensiones después del filtrado por ubicación: {df.shape}")

# --- Paso 2: Inspección de valores nulos en el subconjunto filtrado ---
print("\n--- Valores Nulos ANTES de la Imputación (en el subconjunto filtrado) ---")
null_counts_before = df.isnull().sum()
null_percentages_before = (df.isnull().sum() / len(df)) * 100
missing_data_summary_before = pd.DataFrame({
    'Null Count': null_counts_before,
    'Percentage (%)': null_percentages_before
})
print(missing_data_summary_before[missing_data_summary_before['Null Count'] > 0].sort_values(by='Null Count', ascending=False))

# --- Paso 3: Imputación de Valores Nulos ---
# Identificar columnas numéricas y categóricas (excluyendo 'Date' por ahora)
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

# 'Location' ya está filtrada y no debería tener nulos ni necesitar imputación aquí.
if 'Location' in categorical_cols:
    categorical_cols.remove('Location')
# 'Date' se manejará por separado y no se imputa con estas estrategias.
if 'Date' in df.columns:
    if 'Date' in numerical_cols: numerical_cols.remove('Date') # Aunque 'Date' no es numérico para imputar así
    if 'Date' in categorical_cols: categorical_cols.remove('Date')


# Imputador para columnas numéricas: media
if numerical_cols: # Solo si hay columnas numéricas que imputar
    num_imputer = SimpleImputer(strategy='mean')
    df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

# Imputador para columnas categóricas: moda (valor más frecuente)
cols_to_impute_cat = [col for col in categorical_cols if df[col].isnull().any()]
if cols_to_impute_cat:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df[cols_to_impute_cat] = cat_imputer.fit_transform(df[cols_to_impute_cat])

print("\n--- Valores Nulos DESPUÉS de la Imputación ---")
null_counts_after = df.isnull().sum()
remaining_nulls = null_counts_after[null_counts_after > 0]
if not remaining_nulls.empty:
    print("Columnas con nulos restantes:")
    print(remaining_nulls)
else:
    print("Todos los valores nulos (excepto potencialmente en 'Date' si tuviera) han sido imputados exitosamente.")

print("\n--- Información del Dataset Después de Filtrado e Imputación ---")
df.info()
print(f"\nDimensiones después de la limpieza: {df.shape}")
df.head()

### Celda 3: Información General, Filtrado y Manejo de Nulos Mejorado

**Propósito:** Entender la estructura inicial del dataset, filtrar los datos para las ubicaciones de interés y manejar los valores faltantes de una manera más robusta que simplemente eliminando filas.

**Detalles del Código:**
1.  **Información General del Dataset Completo:** Muestra `info()` y `shape` del DataFrame original.
2.  **Filtrado por Ubicaciones de Interés:** Crea el DataFrame `df` con datos solo de 'Melbourne', 'MelbourneAirport', 'Watsonia'.
3.  **Inspección de Valores Nulos (Subconjunto Filtrado):** Muestra cuentas y porcentajes de nulos *antes* de la imputación.
4.  **Imputación de Valores Nulos:**
    *   Las columnas numéricas se imputan con la media (`SimpleImputer(strategy='mean')`).
    *   Las columnas categóricas (excepto 'Location' y 'Date') se imputan con la moda (`SimpleImputer(strategy='most_frequent')`).
5.  **Verificación Post-Imputación:** Se muestra `info()`, `shape`, y las primeras filas del DataFrame `df` limpio y filtrado.

**Resultado Esperado:**
*   Información sobre el dataset original.
*   Dimensiones antes y después del filtrado por ubicación.
*   Un resumen de los valores nulos antes y después de la imputación.
*   Información del DataFrame `df` después de la imputación, idealmente sin valores nulos (excepto 'Date' si tuviera originalmente, lo cual es improbable para esta columna).
*   Las primeras filas del DataFrame `df` limpio y filtrado.

In [ ]:
# --- Ingeniería de Características a partir de la Columna 'Date' ---

# Convertir la columna 'Date' a formato datetime
# Es posible que 'Date' tenga nulos si el CSV original los tuviera y no se filtraran por ubicación.
# Si 'Date' tiene nulos, pd.to_datetime los convertirá a NaT (Not a Time).
# Estos NaT se propagarán a las columnas derivadas (Year, Month, etc.) como NaN (para flotantes) o NaT.
# Si hay NaTs, se pueden eliminar esas filas o imputar la fecha si es crucial.
# Dado que el dropna() original era agresivo, es posible que no queden NaTs en 'Date'
# después de la imputación general (aunque 'Date' fue excluida de la imputación explícita).
# Verificamos y manejamos si es necesario.
if df['Date'].isnull().any():
    print(f"Valores nulos encontrados en 'Date': {df['Date'].isnull().sum()}. Eliminando estas filas.")
    df.dropna(subset=['Date'], inplace=True)

df['Date'] = pd.to_datetime(df['Date'])

# Extraer componentes de la fecha
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear # Nueva característica: día del año

# Definir función para determinar la estación del año (Hemisferio Sur)
def date_to_season(date_obj):
    month = date_obj.month
    if month in [12, 1, 2]:
        return 'Verano'  # Summer
    elif month in [3, 4, 5]:
        return 'Otoño'   # Autumn
    elif month in [6, 7, 8]:
        return 'Invierno'# Winter
    else: # 9, 10, 11
        return 'Primavera' # Spring

# Aplicar la función para crear la columna 'Season'
df['Season'] = df['Date'].apply(date_to_season)

print("\nNuevas características de fecha y estación añadidas:")
df[['Date', 'Year', 'Month', 'Day', 'DayOfYear', 'Season']].head()

### Celda 4: Ingeniería de Características (Fecha y Estación)

**Propósito:** Transformar la columna 'Date' y extraer información útil como el año, mes, día y la estación del año, lo cual puede ser relevante para el análisis climático.

**Detalles del Código:**
1.  **Manejo de Nulos en 'Date':** Se verifica si hay valores nulos en la columna 'Date' (que se convertirían en `NaT` por `pd.to_datetime`). Si existen, esas filas se eliminan.
2.  **Conversión de 'Date':** `df['Date']` se convierte a tipo `datetime`.
3.  **Extracción de Componentes de Fecha:** Se crean nuevas columnas: 'Year', 'Month', 'Day', y 'DayOfYear'.
4.  **Determinación de la Estación:** Se define y aplica la función `date_to_season` para crear la columna 'Season' (Verano, Otoño, Invierno, Primavera) para el Hemisferio Sur.

**Resultado Esperado:**
*   Un mensaje si se eliminaron filas debido a nulos en 'Date'.
*   Un mensaje indicando que las nuevas características han sido añadidas.
*   Una tabla mostrando las primeras cinco filas de las columnas 'Date', 'Year', 'Month', 'Day', 'DayOfYear', y 'Season'.

In [ ]:
# --- Visualización: Temperatura Máxima Promedio por Mes y Ubicación ---
plt.figure(figsize=(12, 7))
sns.lineplot(data=df, x='Month', y='MaxTemp', hue='Location', estimator='mean', errorbar='sd', lw=2)
plt.title('Temperatura Máxima Promedio Mensual por Ubicación (+/- Desv. Est.)', fontsize=16)
plt.xlabel('Mes del Año', fontsize=12)
plt.ylabel('Temperatura Máxima Promedio (°C)', fontsize=12)
plt.xticks(ticks=range(1, 13), labels=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.legend(title='Ubicación')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Celda 5: EDA - Temperatura Máxima Promedio por Mes y Ubicación

**Propósito:** Explorar cómo varía la temperatura máxima promedio a lo largo de los meses para las diferentes ubicaciones seleccionadas.

**Detalles del Código:**
*   `sns.lineplot` crea un gráfico de líneas mostrando la `MaxTemp` promedio (`estimator='mean'`) por `Month` para cada `Location`.
*   `errorbar='sd'` muestra la desviación estándar como una banda de error.
*   Se personalizan etiquetas, título, y las marcas del eje X para mostrar nombres de meses.

**Resultado Esperado:**
Un gráfico de líneas con tres curvas (una para cada ubicación), representando la tendencia de la temperatura máxima promedio mensual, con bandas de error.

In [ ]:
# --- EDA: Distribución de la Variable Objetivo 'RainTomorrow' ---
# Es importante entender el balance de clases antes de la modelización

# Asegurarse de que 'RainTomorrow' existe y no tiene solo NaNs (ya imputamos)
if 'RainTomorrow' in df.columns:
    plt.figure(figsize=(8, 5))
    
    # Determinar el tipo de 'RainTomorrow' para el conteo y porcentajes
    if df['RainTomorrow'].dtype == 'object': # Asume 'Yes'/'No'
        sns.countplot(data=df, x='RainTomorrow', palette='viridis')
        counts = df['RainTomorrow'].value_counts()
        percentages = df['RainTomorrow'].value_counts(normalize=True) * 100
        class_labels = counts.index.tolist() # ['No', 'Yes'] o viceversa
    elif pd.api.types.is_numeric_dtype(df['RainTomorrow']): # Asume 0/1
        # Mapear temporalmente para etiquetas en el gráfico si es numérico
        temp_rain_tomorrow_labels = df['RainTomorrow'].map({0:'No', 1:'Yes'})
        sns.countplot(x=temp_rain_tomorrow_labels, order=['No','Yes'], palette='viridis')
        counts = temp_rain_tomorrow_labels.value_counts()
        percentages = temp_rain_tomorrow_labels.value_counts(normalize=True) * 100
        class_labels = counts.index.tolist()
    else:
        print("Tipo de 'RainTomorrow' no reconocido para countplot.")
        counts, percentages, class_labels = None, None, None

    if counts is not None:
        plt.title('Distribución de la Variable Objetivo: RainTomorrow', fontsize=15)
        plt.xlabel('¿Lloverá Mañana?', fontsize=12)
        plt.ylabel('Frecuencia', fontsize=12)

        # Ajustar el bucle de anotaciones para que coincida con el orden del countplot
        # El orden de sns.countplot puede no ser el mismo que el de value_counts().index
        # Por ello, iteramos sobre los patches (barras) del gráfico
        ax = plt.gca()
        for i, p in enumerate(ax.patches):
            # El label de la barra actual (ej. 'No' o 'Yes')
            current_label = class_labels[i] if ax.get_xticklabels()[i].get_text() in class_labels else ax.get_xticklabels()[i].get_text()

            # Asegurar que el current_label exista en counts y percentages
            if current_label in counts.index:
                count_val = counts[current_label]
                percentage_val = percentages[current_label]
                ax.text(p.get_x() + p.get_width()/2., p.get_height() + 30, f'{count_val}\n({percentage_val:.1f}%)',
                        ha='center', va='bottom', fontsize=10, color='black')
            else: # Fallback si el label no coincide, menos probable con el ajuste
                ax.text(p.get_x() + p.get_width()/2., p.get_height() + 30, f'{p.get_height():.0f}',
                        ha='center', va='bottom', fontsize=10, color='black')
        plt.show()

        if percentages.min() < 30: # Umbral arbitrario para desbalance
            print(f"Advertencia: La variable 'RainTomorrow' parece desbalanceada: \n{percentages}")
            imbalanced_target = True
        else:
            imbalanced_target = False
    else:
        imbalanced_target = False # Default
else:
    print("La columna 'RainTomorrow' no se encuentra en el DataFrame.")
    imbalanced_target = False # Asumir que no es un problema si no existe

### Celda 6: EDA - Distribución de la Variable Objetivo ('RainTomorrow')

**Propósito:** Analizar la distribución de la variable objetivo `RainTomorrow` para identificar desbalance de clases.

**Detalles del Código:**
*   Se utiliza `sns.countplot` para visualizar la frecuencia de 'Yes' y 'No' (o 0 y 1).
*   El código se adapta si `RainTomorrow` es de tipo `object` o numérico.
*   Se calculan y muestran porcentajes en las barras del gráfico.
*   Se establece una bandera `imbalanced_target` si una clase representa menos del 30% del total, lo cual es útil para estrategias de modelado posteriores.

**Resultado Esperado:**
Un gráfico de barras mostrando la frecuencia y porcentaje de cada clase de `RainTomorrow`. Un mensaje de advertencia si se detecta desbalance.

In [ ]:
# --- EDA: Matriz de Correlación de Características Numéricas ---
# Usamos df que ya tiene las características de fecha procesadas.
numerical_features_for_corr = df.select_dtypes(include=np.number).columns.tolist()
# Excluir coordenadas si no se quieren en la correlación principal, o día/mes/año si se prefiere season
cols_to_exclude_from_corr = ['Year', 'Month', 'Day'] # Lat/Lon no están en 'df' aún.
numerical_features_for_corr = [col for col in numerical_features_for_corr if col not in cols_to_exclude_from_corr]

if numerical_features_for_corr:
    correlation_matrix = df[numerical_features_for_corr].corr()

    plt.figure(figsize=(18, 14)) # Aumentado tamaño para más variables
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, annot_kws={"size": 8})
    plt.title('Matriz de Correlación de Características Numéricas', fontsize=18)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout() # Ajusta el layout para que no se corten las etiquetas
    plt.show()
else:
    print("No hay características numéricas para generar la matriz de correlación.")

### Celda 7: EDA - Matriz de Correlación

**Propósito:** Visualizar las correlaciones lineales entre las características numéricas.

**Detalles del Código:**
*   Se seleccionan las columnas numéricas del DataFrame `df`.
*   Se excluyen 'Year', 'Month', 'Day' de la matriz principal de correlación.
*   Se calcula la matriz de correlación (`.corr()`).
*   Se visualiza usando `sns.heatmap` con anotaciones y una paleta de colores `coolwarm`.

**Resultado Esperado:**
Un mapa de calor mostrando los coeficientes de correlación entre pares de variables numéricas. Ayuda a identificar multicolinealidad.

In [ ]:
# --- Visualización: Distribución de Temperatura Máxima por Estación y Ubicación ---
plt.figure(figsize=(14, 8))
sns.boxplot(data=df, x='Season', y='MaxTemp', hue='Location', palette='viridis',
            order=['Verano', 'Otoño', 'Invierno', 'Primavera']) # Asegurar orden de estaciones
plt.title('Distribución de Temperatura Máxima por Estación y Ubicación', fontsize=16)
plt.xlabel('Estación del Año', fontsize=12)
plt.ylabel('Temperatura Máxima (°C)', fontsize=12)
plt.legend(title='Ubicación', loc='upper right')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Celda 8: EDA - Distribución de Temperatura Máxima por Estación y Ubicación

**Propósito:** Analizar cómo se distribuye `MaxTemp` para cada estación y ubicación usando diagramas de caja.

**Detalles del Código:**
*   `sns.boxplot` crea los diagramas de caja.
*   `x='Season'`, `y='MaxTemp'`, `hue='Location'`.
*   `order` se usa para asegurar el orden cronológico de las estaciones en el eje X.

**Resultado Esperado:**
Un conjunto de diagramas de caja mostrando la distribución de `MaxTemp` (mediana, cuartiles, outliers) para cada combinación de estación y ubicación.

In [ ]:
# --- Visualización: Lluvia Acumulada Mensual por Ubicación ---
# Agrupar por ubicación y mes, y sumar la lluvia
rain_summary_monthly = df.groupby(['Location', 'Month'])['Rainfall'].sum().reset_index()

plt.figure(figsize=(12, 7))
sns.lineplot(data=rain_summary_monthly, x='Month', y='Rainfall', hue='Location', marker='o', lw=2)
plt.title('Lluvia Total Acumulada Mensual por Ubicación', fontsize=16)
plt.xlabel('Mes del Año', fontsize=12)
plt.ylabel('Lluvia Total Acumulada (mm)', fontsize=12)
plt.xticks(ticks=range(1, 13), labels=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.legend(title='Ubicación')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Celda 9: EDA - Lluvia Acumulada Mensual por Ubicación

**Propósito:** Analizar y comparar la cantidad total de lluvia acumulada cada mes para las diferentes ubicaciones.

**Detalles del Código:**
*   Los datos se agrupan por `Location` y `Month`, y se suma `Rainfall`.
*   `sns.lineplot` visualiza esta lluvia acumulada mensual.

**Resultado Esperado:**
Un gráfico de líneas mostrando la tendencia de la lluvia total acumulada a lo largo de los meses para cada ubicación.

In [ ]:
# --- Visualización: Heatmap de Temperatura Máxima Promedio por Mes y Ciudad ---
# Crear tabla pivote: Meses como índice, Ubicaciones como columnas, Temperatura Máxima promedio como valores
pivot_temp_avg = df.pivot_table(index='Month', columns='Location', values='MaxTemp', aggfunc='mean')

# Ordenar el índice (meses) para que aparezcan en orden cronológico
pivot_temp_avg = pivot_temp_avg.sort_index()
# Mapear números de mes a nombres para mejor visualización en el eje Y
month_names = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
pivot_temp_avg.index = [month_names[i-1] for i in pivot_temp_avg.index]


plt.figure(figsize=(10, 8))
sns.heatmap(pivot_temp_avg, annot=True, cmap='YlOrRd', fmt=".1f", linewidths=.5, cbar_kws={'label': 'Temperatura Máxima Promedio (°C)'})
plt.title("Temperatura Máxima Promedio (°C) por Mes y Ubicación", fontsize=16)
plt.xlabel("Ubicación", fontsize=12)
plt.ylabel("Mes del Año", fontsize=12)
plt.yticks(rotation=0) # Asegurar que las etiquetas de los meses sean horizontales
plt.tight_layout()
plt.show()

### Celda 10: EDA - Heatmap de Temperatura Máxima Promedio por Mes y Ubicación

**Propósito:** Presentar de forma concisa la temperatura máxima promedio para cada combinación de mes y ubicación.

**Detalles del Código:**
*   `df.pivot_table` crea una tabla con meses como filas, ubicaciones como columnas, y `MaxTemp` promedio como valores.
*   Los meses se ordenan y se etiquetan con nombres.
*   `sns.heatmap` visualiza esta tabla pivote.

**Resultado Esperado:**
Un mapa de calor donde cada celda coloreada representa la `MaxTemp` promedio para un mes y ubicación específicos.

In [ ]:
# --- Visualización: Humedad Promedio a las 3PM por Estación y Ubicación ---
plt.figure(figsize=(12, 7))

#Convertir 'Season' a un tipo de datos categórico con orden específico
season_order = ['Verano', 'Otoño', 'Invierno', 'Primavera']
df['Season'] = pd.Categorical(df['Season'], categories=season_order, ordered=True)

# Ahora el lineplot respetará el orden de las categorías
sns.lineplot(data=df, x='Season', y='Humidity3pm', hue='Location', estimator='mean', 
             errorbar='sd', marker='o', lw=2)

### Celda 11: EDA - Humedad Promedio a las 3PM por Estación del Año y Ubicación

**Propósito:** Analizar cómo varía la `Humidity3pm` promedio a lo largo de las estaciones para cada ubicación.

**Detalles del Código:**
*   `sns.lineplot` muestra la `Humidity3pm` promedio por `Season` y `Location`.
*   `order` se usa para el eje X de estaciones.

**Resultado Esperado:**
Un gráfico de líneas mostrando la humedad promedio a las 3 PM a través de las cuatro estaciones para cada ubicación.

In [ ]:
# --- Preparación de Datos Geográficos ---

# Coordenadas aproximadas para las ubicaciones de interés
coords = {
    'Melbourne': (-37.8136, 144.9631),
    'MelbourneAirport': (-37.6733, 144.8433),
    'Watsonia': (-37.7167, 145.0833),
}

# Crear un DataFrame con las coordenadas
coord_df = pd.DataFrame.from_dict(coords, orient='index', columns=['Lat', 'Lon']).reset_index()
coord_df = coord_df.rename(columns={'index': 'Location'}) # Evitar inplace=True

# Fusionar las coordenadas con el DataFrame principal 'df'
df_geo = pd.merge(df, coord_df, on='Location', how='left')

# Crear un GeoDataFrame a partir del DataFrame fusionado
try:
    gdf = gpd.GeoDataFrame(
        df_geo,
        geometry=gpd.points_from_xy(df_geo['Lon'], df_geo['Lat']),
        crs="EPSG:4326" # WGS 84
    )
    print("GeoDataFrame creado exitosamente.")
    gdf.head()
except Exception as e:
    print(f"Error al crear GeoDataFrame: {e}")
    gdf = None # Establecer gdf a None si falla la creación

### Celda 12: Preparación de Datos Geográficos

**Propósito:** Enriquecer el conjunto de datos con coordenadas geoespaciales y convertirlo en un `GeoDataFrame`.

**Detalles del Código:**
1.  Se define un diccionario `coords` con latitud y longitud para las ubicaciones.
2.  Se crea `coord_df` a partir de este diccionario.
3.  `df_geo` se crea fusionando `df` con `coord_df` por 'Location'.
4.  `gdf` (GeoDataFrame) se crea a partir de `df_geo`, especificando la geometría a partir de 'Lon' y 'Lat', y el CRS (Sistema de Referencia de Coordenadas) como EPSG:4326.

**Resultado Esperado:**
Un mensaje de éxito y las primeras filas del `GeoDataFrame` `gdf`, que ahora incluye una columna 'geometry'.

In [ ]:
# --- Visualización Geográfica: Mapa Interactivo con Marcadores ---

if gdf is not None:
    # Calcular la temperatura máxima promedio y el total de lluvia por ubicación
    # Para DaysWithRain, necesitamos asegurar que RainToday sea del tipo correcto para la lambda
    # Si RainToday ya fue mapeada a 0/1 en `df` (y por ende en `gdf`), la lambda debe cambiar.
    # Revisando el flujo, RainToday en `df` (y `gdf`) aún es Yes/No.
    
    location_summary = gdf.groupby('Location').agg(
        Lat=('Lat', 'first'),
        Lon=('Lon', 'first'),
        AvgMaxTemp=('MaxTemp', 'mean'),
        TotalRainfall=('Rainfall', 'sum'),
        DaysWithRain=('RainToday', lambda x: (x == 'Yes').sum()) 
    ).reset_index()

    # Crear un mapa base centrado aproximadamente en Melbourne
    map_center = [location_summary['Lat'].mean(), location_summary['Lon'].mean()]
    m = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron")

    # Añadir marcadores para cada ubicación
    for _, row in location_summary.iterrows():
        popup_html = f"""
        <b>Ubicación:</b> {row['Location']}<br>
        <b>Temp. Máx Promedio:</b> {row['AvgMaxTemp']:.1f}°C<br>
        <b>Lluvia Total Acumulada:</b> {row['TotalRainfall']:.1f} mm<br>
        <b>Días con Lluvia:</b> {row['DaysWithRain']}
        """
        iframe = folium.IFrame(html=popup_html, width=250, height=100)
        popup = folium.Popup(iframe, max_width=250)
        
        folium.Marker(
            location=[row['Lat'], row['Lon']],
            popup=popup,
            tooltip=f"{row['Location']}: {row['AvgMaxTemp']:.1f}°C",
            icon=folium.Icon(color="blue", icon="cloud", prefix='fa')
        ).add_to(m)

    print("Mapa con marcadores generado. Se mostrará a continuación:")
    display(m)
else:
    print("GeoDataFrame no está disponible, no se puede generar el mapa.")

### Celda 13: Mapa Interactivo con Folium (Marcadores)

**Propósito:** Visualizar las ubicaciones en un mapa interactivo con información climática resumida.

**Detalles del Código:**
*   Se agrupan datos del `gdf` por `Location` para calcular `AvgMaxTemp`, `TotalRainfall`, y `DaysWithRain`.
*   Se crea un mapa `folium.Map` centrado en la región.
*   Se itera sobre `location_summary` para añadir un `folium.Marker` para cada ubicación, con un `popup` HTML formateado y un `tooltip`.

**Resultado Esperado:**
Un mapa interactivo de Folium con marcadores para cada ubicación. Los popups mostrarán estadísticas climáticas al hacer clic.

In [ ]:
# --- Visualización Interactiva: Relación MinTemp y MaxTemp por Mes (Animada) ---
if gdf is not None:
    # Subconjunto para animación (evitar demasiados frames y datos)
    # Filtrar por años más recientes para una animación más manejable
    df_anim = gdf[gdf['Year'] >= gdf['Year'].max() - 2].copy() # Últimos 3 años aprox.
    
    if not df_anim.empty:
        df_anim['DateStr'] = df_anim['Date'].dt.strftime('%Y-%m-%d') # Para animation_group

        fig_anim = px.scatter(
            df_anim,
            x="MinTemp",
            y="MaxTemp",
            color="Location",
            animation_frame="Month", # Animar por mes
            animation_group="DateStr", # Agrupar por fecha para transiciones suaves
            size="Rainfall",       # El tamaño del punto representa la cantidad de lluvia
            hover_name="Location", # Información al pasar el cursor
            title="Relación MinTemp vs MaxTemp (Animado por Mes, Tamaño por Lluvia)",
            labels={"MinTemp": "Temperatura Mínima (°C)", "MaxTemp": "Temperatura Máxima (°C)", "Rainfall": "Lluvia (mm)"},
            size_max=30,           # Tamaño máximo de las burbujas
            height=700,
            # Para asegurar que la animación se repita y los ejes se mantengan consistentes:
            range_x=[df_anim['MinTemp'].min()-2, df_anim['MinTemp'].max()+2],
            range_y=[df_anim['MaxTemp'].min()-2, df_anim['MaxTemp'].max()+2]
        )

        # Mejorar el layout de la animación
        fig_anim.update_layout(
            transition={'duration': 500}, # Duración de la transición entre frames
            xaxis_title="Temperatura Mínima (°C)",
            yaxis_title="Temperatura Máxima (°C)",
            legend_title_text='Ubicación'
        )
        # Ajustar la velocidad de la animación (más bajo es más rápido)
        if fig_anim.layout.updatemenus: # Comprobar si existen updatemenus
            fig_anim.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000 # Duración por frame en ms
            fig_anim.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300 # Duración de transición

        print("Generando gráfico animado. Esto puede tardar unos momentos...")
        fig_anim.show()
    else:
        print("No hay datos suficientes para la animación después del filtrado por año.")
else:
    print("GeoDataFrame no está disponible para esta visualización.")

### Celda 14: Animación Interactiva con Plotly Express

**Propósito:** Crear una visualización animada de `MinTemp` vs `MaxTemp`, con el tamaño de los puntos representando `Rainfall`, animada por `Month`.

**Detalles del Código:**
*   Se crea un subconjunto `df_anim` con los datos de los últimos 3 años para la animación.
*   `px.scatter` genera el gráfico animado.
    *   `animation_frame="Month"` y `animation_group="DateStr"`.
    *   `size="Rainfall"` y `color="Location"`.
*   Se ajustan los rangos de los ejes y la velocidad/transición de la animación.

**Resultado Esperado:**
Un gráfico de dispersión animado e interactivo de Plotly, que permite explorar la relación entre MinTemp, MaxTemp y Rainfall a lo largo de los meses.

In [ ]:
# --- Preparación de Datos para Modelado de Clasificación (Predecir 'RainTomorrow') ---
if gdf is not None:
    df_ml = gdf.copy() # Usar el GeoDataFrame que tiene toda la información

    # --- 1. Selección de Características (Features) y Variable Objetivo (Target) ---
    features_cls = ['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity9am', 'Humidity3pm',
                    'WindGustSpeed', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm',
                    'WindSpeed9am', 'WindSpeed3pm', 'DayOfYear', 
                    'RainToday', 'Season', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
    target_cls = 'RainTomorrow'

    # Asegurarse de que todas las features y el target existen
    existing_features_cls = [col for col in features_cls if col in df_ml.columns]
    if len(existing_features_cls) != len(features_cls):
        missing_in_features_cls = list(set(features_cls) - set(existing_features_cls))
        print(f"Advertencia: Las siguientes características no se encuentran en df_ml y serán excluidas: {missing_in_features_cls}")
        features_cls = existing_features_cls
        
    if target_cls not in df_ml.columns:
        print(f"ERROR CRÍTICO: La variable objetivo '{target_cls}' no existe. No se puede continuar con la clasificación.")
        X_cls, y_cls = None, None 
    elif not features_cls:
        print(f"ERROR CRÍTICO: No hay características válidas para la clasificación.")
        X_cls, y_cls = None, None
    else:
        X_cls = df_ml[features_cls]
        y_cls = df_ml[target_cls]

    if X_cls is not None and y_cls is not None:
        # --- 2. Codificación de Variables Categóricas y Binarias ---
        y_cls = y_cls.map({'No': 0, 'Yes': 1}).astype(int)
        
        if 'RainToday' in X_cls.columns and X_cls['RainToday'].dtype == 'object':
             X_cls.loc[:, 'RainToday'] = X_cls['RainToday'].map({'No': 0, 'Yes': 1}).astype(int)

        # --- 3. Identificar Columnas Numéricas y Categóricas para el Preprocesador ---
        numerical_cols_cls = X_cls.select_dtypes(include=np.number).columns.tolist()
        categorical_cols_cls = X_cls.select_dtypes(include='object').columns.tolist()
        
        print(f"Características numéricas para clasificación: {numerical_cols_cls}")
        print(f"Características categóricas para clasificación: {categorical_cols_cls}")

        # --- 4. Definir el Preprocesador con ColumnTransformer ---
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())])

        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

        preprocessor_cls = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numerical_cols_cls),
                ('cat', categorical_transformer, categorical_cols_cls)], 
            remainder='passthrough')
        
        # --- 5. División de Datos en Entrenamiento y Prueba ---
        X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
            X_cls, y_cls,
            test_size=0.25,
            random_state=42,
            stratify=y_cls)

        print(f"Dimensiones de X_train_cls: {X_train_cls.shape}, y_train_cls: {y_train_cls.shape}")
        print(f"Dimensiones de X_test_cls: {X_test_cls.shape}, y_test_cls: {y_test_cls.shape}")
        print(f"Distribución de 'RainTomorrow' en y_train_cls:\n{y_train_cls.value_counts(normalize=True)}")
    else:
        # Asegurar que las variables se definan como None si falla la preparación
        X_train_cls, X_test_cls, y_train_cls, y_test_cls, preprocessor_cls = [None]*5
        print("No se pudo preparar X_cls o y_cls. Revise los mensajes de error.")
else:
    print("GeoDataFrame no está disponible, no se puede preparar datos para ML.")
    X_cls, y_cls, X_train_cls, X_test_cls, y_train_cls, y_test_cls, preprocessor_cls = [None]*7

### Celda 15: Preparación de Datos para Modelado de Clasificación (`RainTomorrow`)

**Propósito:** Preparar los datos para entrenar modelos que predigan `RainTomorrow`.

**Detalles del Código:**
1.  `df_ml` se crea como copia de `gdf`.
2.  Se definen `features_cls` y `target_cls`. Se verifica su existencia.
3.  `RainTomorrow` (objetivo) y `RainToday` (característica) se mapean de 'Yes'/'No' a 1/0. Se usa `.loc` para la asignación en `X_cls` para evitar `SettingWithCopyWarning`.
4.  Se identifican columnas numéricas y categóricas para el `ColumnTransformer`.
5.  Se define `preprocessor_cls` con transformadores para escalar datos numéricos y aplicar One-Hot Encoding a categóricos, incluyendo imputación como salvaguarda.
6.  Los datos se dividen en conjuntos de entrenamiento y prueba (`train_test_split`) con `stratify=y_cls` para manejar el desbalance de clases.

**Resultado Esperado:**
Mensajes sobre las características, las dimensiones de los conjuntos de datos de entrenamiento/prueba, y la distribución de `RainTomorrow` en el conjunto de entrenamiento.

In [ ]:
# --- Modelado de Clasificación: RandomForest y LogisticRegression con Optimización ---

# Verificar que los datos de entrenamiento estén disponibles y el preprocesador exista
if 'X_train_cls' in globals() and X_train_cls is not None and \
   'y_train_cls' in globals() and y_train_cls is not None and \
   'preprocessor_cls' in globals() and preprocessor_cls is not None:

    # --- Modelo 1: Logistic Regression ---
    print("--- Entrenando Regresión Logística ---")
    # Determinar class_weight basado en imbalanced_target (definido en Celda 6)
    lr_class_weight = None
    if 'imbalanced_target' in globals() and imbalanced_target:
        lr_class_weight = 'balanced'

    pipeline_lr = Pipeline(steps=[
        ('preprocessor', preprocessor_cls),
        ('classifier', LogisticRegression(solver='liblinear', random_state=42,
                                          class_weight=lr_class_weight))
    ])

    param_grid_lr = {
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__penalty': ['l1', 'l2']
    }
    # StratifiedKFold es bueno para clasificación, especialmente con desbalance
    cv_stratified = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, cv=cv_stratified, scoring='roc_auc', n_jobs=-1, verbose=0)
    grid_search_lr.fit(X_train_cls, y_train_cls)

    print(f"Mejores hiperparámetros para Regresión Logística: {grid_search_lr.best_params_}")
    print(f"Mejor ROC AUC (CV) para Regresión Logística: {grid_search_lr.best_score_:.4f}")
    best_lr_model = grid_search_lr.best_estimator_
    y_pred_lr = best_lr_model.predict(X_test_cls)
    y_proba_lr = best_lr_model.predict_proba(X_test_cls)[:, 1]

    # --- Modelo 2: Random Forest Classifier ---
    print("\n--- Entrenando Random Forest Classifier ---")
    rf_class_weight = None
    if 'imbalanced_target' in globals() and imbalanced_target:
        rf_class_weight = 'balanced'

    # Renombrar pipeline y grid_search para evitar colisiones con RandomForestRegressor
    pipeline_rf_cls = Pipeline(steps=[
        ('preprocessor', preprocessor_cls),
        ('classifier', RandomForestClassifier(random_state=42,
                                              class_weight=rf_class_weight))
    ])

    # Hiperparámetros para Random Forest
    param_grid_rf_cls = { # Renombrado
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5],
    }

    grid_search_rf_cls = GridSearchCV(pipeline_rf_cls, param_grid_rf_cls, cv=cv_stratified, scoring='roc_auc', n_jobs=-1, verbose=0)
    grid_search_rf_cls.fit(X_train_cls, y_train_cls)

    print(f"Mejores hiperparámetros para Random Forest: {grid_search_rf_cls.best_params_}")
    print(f"Mejor ROC AUC (CV) para Random Forest: {grid_search_rf_cls.best_score_:.4f}")
    best_rf_cls_model = grid_search_rf_cls.best_estimator_ # Renombrado
    y_pred_rf_cls = best_rf_cls_model.predict(X_test_cls) # Renombrado
    y_proba_rf_cls = best_rf_cls_model.predict_proba(X_test_cls)[:, 1] # Renombrado

    # --- Evaluación Comparativa ---
    print("\n--- Evaluación en el Conjunto de Prueba ---")
    print("\nRegresión Logística - Reporte de Clasificación:")
    print(classification_report(y_test_cls, y_pred_lr, target_names=['No Lluvia', 'Lluvia']))
    print(f"Regresión Logística - ROC AUC: {roc_auc_score(y_test_cls, y_proba_lr):.4f}")

    print("\nRandom Forest - Reporte de Clasificación:")
    print(classification_report(y_test_cls, y_pred_rf_cls, target_names=['No Lluvia', 'Lluvia']))
    print(f"Random Forest - ROC AUC: {roc_auc_score(y_test_cls, y_proba_rf_cls):.4f}")

    # --- Matriz de Confusión ---
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ConfusionMatrixDisplay.from_estimator(best_lr_model, X_test_cls, y_test_cls, ax=axes[0], display_labels=['No Lluvia', 'Lluvia'], cmap='Blues')
    axes[0].set_title('Matriz de Confusión - Regresión Logística')

    ConfusionMatrixDisplay.from_estimator(best_rf_cls_model, X_test_cls, y_test_cls, ax=axes[1], display_labels=['No Lluvia', 'Lluvia'], cmap='Greens')
    axes[1].set_title('Matriz de Confusión - Random Forest')
    plt.tight_layout()
    plt.show()

    # --- Curva ROC ---
    plt.figure(figsize=(8, 6))
    fpr_lr, tpr_lr, _ = roc_curve(y_test_cls, y_proba_lr)
    plt.plot(fpr_lr, tpr_lr, label=f"Regresión Logística (AUC = {roc_auc_score(y_test_cls, y_proba_lr):.2f})")
    
    fpr_rf, tpr_rf, _ = roc_curve(y_test_cls, y_proba_rf_cls)
    plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {roc_auc_score(y_test_cls, y_proba_rf_cls):.2f})")
    
    plt.plot([0, 1], [0, 1], 'k--') # Línea de azar
    plt.xlabel('Tasa de Falsos Positivos (FPR)')
    plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
    plt.title('Curvas ROC Comparativas')
    plt.legend()
    plt.grid(True)
    plt.show()

else:
    print("Los datos de entrenamiento para clasificación (X_train_cls, y_train_cls) o el preprocesador (preprocessor_cls) no están disponibles. Modelado omitido.")
    # Definir variables para evitar errores en celdas posteriores si esta rama se ejecuta
    best_lr_model, best_rf_cls_model = None, None
    y_pred_lr, y_proba_lr, y_pred_rf_cls, y_proba_rf_cls = None, None, None, None

### Celda 16: Modelado de Clasificación (RandomForest y Regresión Logística con `GridSearchCV`)

**Propósito:** Entrenar, optimizar y evaluar modelos de Regresión Logística y Random Forest para predecir `RainTomorrow`.

**Detalles del Código:**
*   Se verifica que las variables necesarias (`X_train_cls`, `y_train_cls`, `preprocessor_cls`) existan.
*   Se definen `Pipeline`s para cada modelo, incluyendo el `preprocessor_cls`.
*   `class_weight='balanced'` se usa si la variable global `imbalanced_target` (definida en Celda 6 de EDA) es `True`.
*   Se definen grillas de hiperparámetros (`param_grid_lr`, `param_grid_rf_cls`).
*   `GridSearchCV` con `StratifiedKFold` (para mantener la proporción de clases en cada fold durante la validación cruzada) y `scoring='roc_auc'` encuentra los mejores modelos. Se renombraron algunas variables de Random Forest Classifier (`pipeline_rf_cls`, `best_rf_cls_model`, etc.) para evitar colisiones con los modelos de regresión que vendrán después.
*   Se evalúan los modelos en el conjunto de prueba (`X_test_cls`, `y_test_cls`) usando `classification_report`, `roc_auc_score`.
*   Se visualizan las matrices de confusión y las curvas ROC comparativas.
*   Si los datos de entrenamiento no están disponibles, se omite el modelado y se definen las variables de modelo y predicción como `None`.

**Resultado Esperado:**
Si los datos están listos:
*   Mejores hiperparámetros y scores de ROC AUC de validación cruzada para cada modelo.
*   Reportes de clasificación detallados (precisión, recall, F1-score) para cada modelo en el conjunto de prueba.
*   Valores de ROC AUC en el conjunto de prueba.
*   Visualizaciones de las matrices de confusión y las curvas ROC.
Si los datos no están listos, un mensaje indicando que se omitió el modelado.

In [ ]:
# --- Importancia de Características para el Mejor Random Forest Classifier ---
if 'best_rf_cls_model' in globals() and best_rf_cls_model is not None and \
   'numerical_cols_cls' in globals() and 'categorical_cols_cls' in globals(): # Asegurar que estas listas existen
    
    preprocessor_fitted_cls = best_rf_cls_model.named_steps['preprocessor']
    classifier_fitted_cls = best_rf_cls_model.named_steps['classifier']

    try:
        # 1. Obtener nombres de características numéricas
        # Directamente de la lista original, ya que StandardScaler no cambia el número ni el orden.
        num_feature_names_cls = numerical_cols_cls

        # 2. Obtener nombres de características categóricas transformadas por OneHotEncoder
        cat_pipeline_cls = preprocessor_fitted_cls.named_transformers_['cat']
        onehot_encoder_cls = cat_pipeline_cls.named_steps['onehot']
        
        # Usar la lista original de columnas categóricas alimentada al ColumnTransformer
        original_cat_cols_cls_from_transformer = categorical_cols_cls

        if original_cat_cols_cls_from_transformer: # Si hay columnas categóricas
             cat_feature_names_cls = list(onehot_encoder_cls.get_feature_names_out(original_cat_cols_cls_from_transformer))
        else:
            cat_feature_names_cls = []
            
        # Combinar todos los nombres de características en el orden correcto
        all_feature_names_cls = list(num_feature_names_cls) + cat_feature_names_cls
        
        importances_cls = classifier_fitted_cls.feature_importances_

        # Verificar que el número de importancias coincida con el número de nombres de características
        if len(all_feature_names_cls) == len(importances_cls):
            feature_importance_cls_df = pd.DataFrame({'feature': all_feature_names_cls, 'importance': importances_cls})
            feature_importance_cls_df = feature_importance_cls_df.sort_values('importance', ascending=False).reset_index(drop=True)

            N = 20 
            plt.figure(figsize=(12, max(6, N*0.35))) 
            sns.barplot(x='importance', y='feature', data=feature_importance_cls_df.head(N), palette='viridis')
            plt.title(f'Top {N} Características Más Importantes (Random Forest Classifier)', fontsize=16)
            plt.xlabel('Importancia (Reducción Media de Impureza)', fontsize=12)
            plt.ylabel('Característica', fontsize=12)
            plt.tight_layout()
            plt.show()
        else:
            print(f"Error: El número de nombres de características ({len(all_feature_names_cls)}) no coincide con el número de importancias ({len(importances_cls)}).")
            print("Nombres de características numéricas:", num_feature_names_cls)
            print("Nombres de características categóricas (transformadas):", cat_feature_names_cls)

    except Exception as e:
        print(f"Error al generar importancia de características para RF Classifier: {e}")
        print("Verifique que 'numerical_cols_cls' y 'categorical_cols_cls' estén correctamente definidos y que el preprocesador se ajustó correctamente.")
else:
    print("El modelo Random Forest Classifier (best_rf_cls_model) no está definido, no se entrenó, o faltan listas de columnas (numerical_cols_cls/categorical_cols_cls).")

### Celda 17: Importancia de Características para Random Forest Classifier

**Propósito:** Visualizar las características más influyentes para el modelo `RandomForestClassifier` optimizado.

**Detalles del Código:**
*   Se verifica que el modelo `best_rf_cls_model` y las listas `numerical_cols_cls` y `categorical_cols_cls` existan.
*   Se accede al preprocesador (`preprocessor_fitted_cls`) y al clasificador (`classifier_fitted_cls`) ajustados desde el pipeline.
*   Se obtienen los nombres de las características numéricas directamente de `numerical_cols_cls`.
*   Para las características categóricas, se utiliza `get_feature_names_out` del `OneHotEncoder` (que está dentro de un pipeline anidado en el `ColumnTransformer`) aplicado a la lista original `categorical_cols_cls`.
*   Se combinan los nombres de las características numéricas y categóricas transformadas.
*   Se extraen las `feature_importances_` del clasificador Random Forest.
*   Se realiza una verificación para asegurar que el número de nombres de características coincida con el número de valores de importancia.
*   Se crea un DataFrame y se visualizan las `N` características más importantes usando un gráfico de barras horizontales.
*   Se incluye un manejo de errores más detallado.

**Resultado Esperado:**
Si el modelo se entrenó correctamente y las listas de columnas están disponibles:
*   Un gráfico de barras mostrando las `N` características más importantes según el modelo Random Forest Classifier para la predicción de `RainTomorrow`.
Si hay problemas, se mostrarán mensajes de error o advertencias.

In [ ]:
# --- Preparación de Datos para Modelado de Regresión (Predecir MaxTemp y Rainfall) ---

# Inicializar variables para evitar errores si gdf no existe o hay problemas
X_train_reg_temp, X_test_reg_temp, y_train_reg_temp, y_test_reg_temp = [None]*4
X_train_reg_rain, X_test_reg_rain, y_train_reg_rain, y_test_reg_rain = [None]*4
preprocessor_reg_temp, preprocessor_reg_rain = None, None
numerical_cols_reg_temp, categorical_cols_reg_temp = [], []
numerical_cols_reg_rain, categorical_cols_reg_rain = [], []
y_train_reg_rain_log = None # Para la transformación log de y_train_reg_rain


if 'gdf' in globals() and gdf is not None:
    df_reg = gdf.copy()

    # --- 1. Codificación Binaria (si es necesario) ---
    if 'RainToday' in df_reg.columns:
        if df_reg['RainToday'].dtype == 'object':
            df_reg.loc[:, 'RainToday'] = df_reg['RainToday'].map({'No': 0, 'Yes': 1}).astype(int)
        elif pd.api.types.is_numeric_dtype(df_reg['RainToday']): # Ensure it's int if already numeric
            df_reg.loc[:, 'RainToday'] = df_reg['RainToday'].astype(int)

    # --- 2. Selección de Características (Features) ---
    intended_base_features_reg = ['MinTemp', 'Humidity9am', 'Humidity3pm', 'WindGustSpeed',
                             'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm',
                             'WindSpeed9am', 'WindSpeed3pm', 'DayOfYear',
                             'RainToday', 'Season', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
    
    # Asegurar que todas las features base existen en df_reg
    base_features_reg = [col for col in intended_base_features_reg if col in df_reg.columns]
    
    # --- Para predecir MaxTemp ---
    target_temp = 'MaxTemp'
    # Start with base features, exclude the target
    features_reg_temp_candidate = [f for f in base_features_reg if f != target_temp]
    # Asegurar que Rainfall (de hoy) esté como predictor para MaxTemp (de hoy)
    if 'Rainfall' not in features_reg_temp_candidate and 'Rainfall' in df_reg.columns:
        features_reg_temp_candidate.append('Rainfall')
    # Filtrar nuevamente para asegurar que todas las características seleccionadas existan en df_reg
    features_reg_temp = list(set(f for f in features_reg_temp_candidate if f in df_reg.columns))

    X_reg_temp, y_reg_temp = None, None
    if target_temp in df_reg.columns and features_reg_temp:
        print(f"Características seleccionadas para regresión (MaxTemp): {features_reg_temp}")
        X_reg_temp = df_reg[features_reg_temp].copy() # Use .copy()
        y_reg_temp = df_reg[target_temp].copy()
    else:
        print(f"ERROR: No se puede preparar datos para regresión de MaxTemp (target '{target_temp}' o features faltantes/vacías).")

    # --- Para predecir Rainfall ---
    target_rain = 'Rainfall'
    features_reg_rain_candidate = [f for f in base_features_reg if f != target_rain]
    if 'MaxTemp' not in features_reg_rain_candidate and 'MaxTemp' in df_reg.columns:
        features_reg_rain_candidate.append('MaxTemp')
    features_reg_rain = list(set(f for f in features_reg_rain_candidate if f in df_reg.columns))
    
    X_reg_rain, y_reg_rain = None, None
    if target_rain in df_reg.columns and features_reg_rain:
        print(f"Características seleccionadas para regresión (Rainfall): {features_reg_rain}")
        X_reg_rain = df_reg[features_reg_rain].copy() # Use .copy()
        y_reg_rain = df_reg[target_rain].copy()
    else:
        print(f"ERROR: No se puede preparar datos para regresión de Rainfall (target '{target_rain}' o features faltantes/vacías).")

    # --- 3 & 4. Definir Preprocesadores y Dividir Datos ---
    reg_numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    reg_categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

    # Preprocesador para MaxTemp
    if X_reg_temp is not None and y_reg_temp is not None:
        numerical_cols_reg_temp = []
        categorical_cols_reg_temp = []
        for col in X_reg_temp.columns:
            if pd.api.types.is_numeric_dtype(X_reg_temp[col]):
                numerical_cols_reg_temp.append(col)
            else:
                categorical_cols_reg_temp.append(col)
        
        print(f"\nPara MaxTemp - Características numéricas: {numerical_cols_reg_temp}")
        print(f"Para MaxTemp - Características categóricas: {categorical_cols_reg_temp}")

        transformers_list_temp = []
        if numerical_cols_reg_temp:
            transformers_list_temp.append(('num', reg_numeric_transformer, numerical_cols_reg_temp))
        if categorical_cols_reg_temp:
            transformers_list_temp.append(('cat', reg_categorical_transformer, categorical_cols_reg_temp))

        if not transformers_list_temp:
            print("ERROR (MaxTemp): No hay transformadores definidos.")
            preprocessor_reg_temp = None
        else:
            preprocessor_reg_temp = ColumnTransformer(
                transformers=transformers_list_temp,
                remainder='drop' # Crucial: drop unhandled columns
            )
            X_train_reg_temp, X_test_reg_temp, y_train_reg_temp, y_test_reg_temp = train_test_split(
                X_reg_temp, y_reg_temp, test_size=0.25, random_state=42)
            print(f"Dimensiones X_train_reg_temp: {X_train_reg_temp.shape if X_train_reg_temp is not None else 'N/A'}")

    # Preprocesador para Rainfall
    if X_reg_rain is not None and y_reg_rain is not None:
        numerical_cols_reg_rain = []
        categorical_cols_reg_rain = []
        for col in X_reg_rain.columns:
            if pd.api.types.is_numeric_dtype(X_reg_rain[col]):
                numerical_cols_reg_rain.append(col)
            else:
                categorical_cols_reg_rain.append(col)

        print(f"\nPara Rainfall - Características numéricas: {numerical_cols_reg_rain}")
        print(f"Para Rainfall - Características categóricas: {categorical_cols_reg_rain}")
        
        transformers_list_rain = []
        if numerical_cols_reg_rain:
            transformers_list_rain.append(('num', reg_numeric_transformer, numerical_cols_reg_rain))
        if categorical_cols_reg_rain:
            transformers_list_rain.append(('cat', reg_categorical_transformer, categorical_cols_reg_rain))

        if not transformers_list_rain:
            print("ERROR (Rainfall): No hay transformadores definidos.")
            preprocessor_reg_rain = None
        else:
            preprocessor_reg_rain = ColumnTransformer(
                transformers=transformers_list_rain,
                remainder='drop' # Crucial: drop unhandled columns
            )
            X_train_reg_rain, X_test_reg_rain, y_train_reg_rain, y_test_reg_rain = train_test_split(
                X_reg_rain, y_reg_rain, test_size=0.25, random_state=42)
            
            if y_train_reg_rain is not None: # Asegurarse que y_train_reg_rain existe antes de transformarlo
                y_train_reg_rain_log = np.log1p(y_train_reg_rain)
            print(f"Dimensiones X_train_reg_rain: {X_train_reg_rain.shape if X_train_reg_rain is not None else 'N/A'}")
else:
    print("GeoDataFrame (gdf) no está disponible, no se puede preparar datos para ML de regresión.")

### Celda 18: Preparación de Datos para Modelado de Regresión (`MaxTemp` y `Rainfall`)

**Propósito:** Preparar los datos para dos tareas de regresión distintas: predecir la temperatura máxima (`MaxTemp`) y la cantidad de lluvia (`Rainfall`).

**Detalles del Código:**
1.  **Inicialización:** Las variables clave para los datos de regresión (`X_train_reg_temp`, `preprocessor_reg_temp`, etc.) se inicializan a `None` o listas vacías. Esto es para evitar errores `NameError` en celdas posteriores si `gdf` no existe o si la preparación de datos para una de las tareas falla.
2.  **Copia y Codificación:** Si `gdf` existe, se crea `df_reg`. La columna `RainToday` se codifica a 0/1 si es de tipo objeto.
3.  **Selección de Características Base:** Se define `base_features_reg` con un conjunto de predictores potenciales. Se filtran para asegurar que solo las columnas existentes en `df_reg` se consideren.
4.  **Preparación para `MaxTemp`:**
    *   `target_temp` es 'MaxTemp'.
    *   `features_reg_temp_candidate` se crea excluyendo `target_temp` de `base_features_reg`.
    *   Se asegura que 'Rainfall' (lluvia de hoy) se incluya como predictor para 'MaxTemp' (temperatura máxima de hoy).
    *   `features_reg_temp` se finaliza filtrando nuevamente para asegurar que todas las características existan.
    *   Se crean `X_reg_temp` e `y_reg_temp` si el target y las features son válidos.
5.  **Preparación para `Rainfall`:**
    *   `target_rain` es 'Rainfall'.
    *   `features_reg_rain_candidate` se crea excluyendo `target_rain`.
    *   Se asegura que 'MaxTemp' (temperatura máxima de hoy) se incluya como predictor para 'Rainfall' (lluvia de hoy).
    *   `features_reg_rain` se finaliza con un filtro de existencia.
    *   Se crean `X_reg_rain` e `y_reg_rain` si son válidos.
6.  **Preprocesadores y División de Datos:**
    *   Se definen transformadores comunes (`reg_numeric_transformer`, `reg_categorical_transformer`).
    *   Si `X_reg_temp` e `y_reg_temp` se crearon con éxito:
        *   Se identifican `numerical_cols_reg_temp` y `categorical_cols_reg_temp`.
        *   Se define `preprocessor_reg_temp`.
        *   Se realiza el `train_test_split`.
    *   Se repite un proceso similar para `X_reg_rain` e `y_reg_rain`, creando `preprocessor_reg_rain` y los conjuntos de datos correspondientes.
    *   **Importante:** `y_train_reg_rain_log` se crea aplicando `np.log1p` a `y_train_reg_rain` aquí, ya que el modelo de lluvia se entrenará con la variable objetivo transformada.

**Resultado Esperado:**
*   Mensajes de estado indicando la preparación de datos para cada tarea de regresión.
*   Listas de características numéricas y categóricas para cada tarea.
*   Dimensiones de los conjuntos de entrenamiento `X_train_reg_temp` y `X_train_reg_rain`.
*   Mensajes de error si los targets o las features no se pueden definir correctamente.

In [ ]:
# --- Modelo de Regresión para MaxTemp con RandomForestRegressor y Optimización ---

# Inicializar métricas y modelo para evitar errores si no se ejecuta
r2_temp, rmse_temp = np.nan, np.nan
best_rf_temp_model = None
y_pred_reg_temp = None # Para el gráfico de predicción vs real

if 'X_train_reg_temp' in globals() and X_train_reg_temp is not None and \
   'y_train_reg_temp' in globals() and y_train_reg_temp is not None and \
   'preprocessor_reg_temp' in globals() and preprocessor_reg_temp is not None:
    
    print("--- Entrenando RandomForestRegressor para MaxTemp ---")
    
    pipeline_rf_temp = Pipeline(steps=[
        ('preprocessor', preprocessor_reg_temp),
        ('regressor', RandomForestRegressor(random_state=42))])

    # Usar un rango de hiperparámetros consistente, puede ser el mismo que el de clasificación de RF si aplica, o ajustado
    param_grid_rf_reg = {
        'regressor__n_estimators': [100, 150],       # Reducido para velocidad en demostración
        'regressor__max_depth': [10, 20, None],      # None para permitir crecimiento completo
        'regressor__min_samples_split': [2, 5],
        'regressor__min_samples_leaf': [1, 2]        # Añadido para regularización
    }
    # KFold normal para regresión (StratifiedKFold es para clasificación)
    cv_kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    
    grid_search_rf_temp = GridSearchCV(pipeline_rf_temp, param_grid_rf_reg, cv=cv_kfold, scoring='r2', n_jobs=-1, verbose=0)
    grid_search_rf_temp.fit(X_train_reg_temp, y_train_reg_temp)

    print(f"Mejores hiperparámetros para RFR (MaxTemp): {grid_search_rf_temp.best_params_}")
    print(f"Mejor R² (CV) para RFR (MaxTemp): {grid_search_rf_temp.best_score_:.4f}")
    
    best_rf_temp_model = grid_search_rf_temp.best_estimator_
    y_pred_reg_temp = best_rf_temp_model.predict(X_test_reg_temp)

    # Métricas de Evaluación
    r2_temp = r2_score(y_test_reg_temp, y_pred_reg_temp)
    rmse_temp = mean_squared_error(y_test_reg_temp, y_pred_reg_temp, squared=False)

    print("\nRandomForestRegressor – Predicción de MaxTemp:")
    print(f"  R² en conjunto de prueba: {r2_temp:.3f}")
    print(f"  RMSE en conjunto de prueba: {rmse_temp:.2f}°C")

    # --- Importancia de Características para RFR (MaxTemp) ---
    if best_rf_temp_model is not None and \
       'numerical_cols_reg_temp' in globals() and 'categorical_cols_reg_temp' in globals():
        try:
            preprocessor_fitted_temp = best_rf_temp_model.named_steps['preprocessor']
            regressor_fitted_temp = best_rf_temp_model.named_steps['regressor']
            
            num_feat_names_temp = numerical_cols_reg_temp
            
            cat_pipeline_temp = preprocessor_fitted_temp.named_transformers_['cat']
            onehot_encoder_temp = cat_pipeline_temp.named_steps['onehot']
            original_cat_cols_temp_transformer = categorical_cols_reg_temp

            if original_cat_cols_temp_transformer:
                 cat_feat_names_temp = list(onehot_encoder_temp.get_feature_names_out(original_cat_cols_temp_transformer))
            else:
                cat_feat_names_temp = []
                
            all_feat_names_temp = list(num_feat_names_temp) + cat_feat_names_temp
            
            importances_temp = regressor_fitted_temp.feature_importances_

            if len(all_feat_names_temp) == len(importances_temp):
                feature_importance_temp_df = pd.DataFrame({'feature': all_feat_names_temp, 'importance': importances_temp})
                feature_importance_temp_df = feature_importance_temp_df.sort_values('importance', ascending=False).reset_index(drop=True)

                N = 15
                plt.figure(figsize=(10, max(5, N*0.35)))
                sns.barplot(x='importance', y='feature', data=feature_importance_temp_df.head(N), palette='mako')
                plt.title(f'Top {N} Características - Predicción MaxTemp (RFR)', fontsize=16)
                plt.xlabel('Importancia', fontsize=12)
                plt.ylabel('Característica', fontsize=12)
                plt.tight_layout()
                plt.show()
            else:
                print(f"Error en feat. imp. (MaxTemp): Nombres ({len(all_feat_names_temp)}) vs Importancias ({len(importances_temp)}).")

        except Exception as e:
            print(f"Error al generar importancia de características para RFR (MaxTemp): {e}")
else:
    print("Los datos de entrenamiento/preprocesador para regresión de MaxTemp no están disponibles. Modelado omitido.")

### Celda 19: Modelo de Regresión para `MaxTemp` (RandomForestRegressor con `GridSearchCV`)

**Propósito:** Entrenar, optimizar y evaluar un modelo `RandomForestRegressor` para predecir la temperatura máxima (`MaxTemp`).

**Detalles del Código:**
1.  **Inicialización:** `r2_temp`, `rmse_temp`, `best_rf_temp_model`, y `y_pred_reg_temp` se inicializan para evitar errores si el modelado se omite.
2.  **Verificación:** Se comprueba que los datos de entrenamiento (`X_train_reg_temp`, `y_train_reg_temp`) y el preprocesador (`preprocessor_reg_temp`) estén disponibles.
3.  **Pipeline y GridSearchCV:**
    *   Se crea una `Pipeline` con el `preprocessor_reg_temp` y un `RandomForestRegressor`.
    *   Se define `param_grid_rf_reg` para los hiperparámetros del regresor (incluyendo `min_samples_leaf` para regularización).
    *   `GridSearchCV` con `KFold` (adecuado para regresión) y `scoring='r2'` se utiliza para encontrar el mejor modelo.
4.  **Evaluación:**
    *   El mejor modelo se usa para predecir en `X_test_reg_temp`.
    *   Se calculan y muestran R² y RMSE (Error Cuadrático Medio Raíz).
5.  **Importancia de Características:**
    *   Se extraen y visualizan las importancias de las características del `RandomForestRegressor` entrenado, de manera similar a como se hizo para el clasificador. Se verifica que las listas `numerical_cols_reg_temp` y `categorical_cols_reg_temp` existan.
    *   Se incluye una comprobación de la longitud de los nombres de características y las importancias.

**Resultado Esperado:**
Si los datos están listos:
*   Mejores hiperparámetros y R² de validación cruzada para el `RandomForestRegressor`.
*   Métricas R² y RMSE en el conjunto de prueba para la predicción de `MaxTemp`.
*   Un gráfico de barras mostrando las características más importantes para predecir `MaxTemp`.
Si los datos no están listos, un mensaje indicando que se omitió el modelado.

In [ ]:
# --- Modelo de Regresión para Rainfall con RandomForestRegressor, Log-Transform y Optimización ---

# Inicializar métricas y modelo
r2_rain, rmse_rain = np.nan, np.nan
best_rf_rain_model = None
y_pred_reg_rain = None # Para el gráfico de predicción vs real

if 'X_train_reg_rain' in globals() and X_train_reg_rain is not None and \
   'y_train_reg_rain_log' in globals() and y_train_reg_rain_log is not None and \
   'y_test_reg_rain' in globals() and y_test_reg_rain is not None and \
   'preprocessor_reg_rain' in globals() and preprocessor_reg_rain is not None:
    
    print("--- Entrenando RandomForestRegressor para Rainfall (con Log-Transform) ---")
    
    # y_train_reg_rain_log ya debería estar calculado en la celda de preparación de datos de regresión.
    
    pipeline_rf_rain = Pipeline(steps=[
        ('preprocessor', preprocessor_reg_rain), # Usar el preprocesador específico para Rainfall
        ('regressor', RandomForestRegressor(random_state=42))
    ])

    # Reutilizar param_grid_rf_reg (definido para MaxTemp RFR) o definir uno nuevo si es necesario
    # Para este ejemplo, lo reutilizamos.
    if 'param_grid_rf_reg' not in globals(): # Definir si no existe globalmente
        param_grid_rf_reg = {
            'regressor__n_estimators': [100, 150],
            'regressor__max_depth': [10, 20, None],
            'regressor__min_samples_split': [2, 5],
            'regressor__min_samples_leaf': [1, 2]
        }
    if 'cv_kfold' not in globals(): # Definir si no existe globalmente
        cv_kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    grid_search_rf_rain = GridSearchCV(pipeline_rf_rain, param_grid_rf_reg, cv=cv_kfold, scoring='r2', n_jobs=-1, verbose=0)
    grid_search_rf_rain.fit(X_train_reg_rain, y_train_reg_rain_log) # Entrenar con y_log

    print(f"Mejores hiperparámetros para RFR (Rainfall): {grid_search_rf_rain.best_params_}")
    print(f"Mejor R² (CV) para RFR (Rainfall en escala log): {grid_search_rf_rain.best_score_:.4f}")
    
    best_rf_rain_model = grid_search_rf_rain.best_estimator_
    y_pred_reg_rain_log = best_rf_rain_model.predict(X_test_reg_rain)

    # Inversión de la transformación logarítmica para obtener predicciones en la escala original
    y_pred_reg_rain = np.expm1(y_pred_reg_rain_log)
    # Asegurar que las predicciones no sean negativas (la lluvia no puede ser negativa)
    y_pred_reg_rain = np.maximum(0, y_pred_reg_rain)

    # Métricas de Evaluación (en la escala original)
    r2_rain = r2_score(y_test_reg_rain, y_pred_reg_rain)
    rmse_rain = mean_squared_error(y_test_reg_rain, y_pred_reg_rain, squared=False)

    print("\nRandomForestRegressor – Predicción de Rainfall (evaluado en escala original):")
    print(f"  R² en conjunto de prueba: {r2_rain:.3f}")
    print(f"  RMSE en conjunto de prueba: {rmse_rain:.2f} mm")

    # --- Importancia de Características para RFR (Rainfall) ---
    if best_rf_rain_model is not None and \
       'numerical_cols_reg_rain' in globals() and 'categorical_cols_reg_rain' in globals():
        try:
            preprocessor_fitted_rain = best_rf_rain_model.named_steps['preprocessor']
            regressor_fitted_rain = best_rf_rain_model.named_steps['regressor']

            num_feat_names_rain = numerical_cols_reg_rain
            
            cat_pipeline_rain = preprocessor_fitted_rain.named_transformers_['cat']
            onehot_encoder_rain = cat_pipeline_rain.named_steps['onehot']
            original_cat_cols_rain_transformer = categorical_cols_reg_rain

            if original_cat_cols_rain_transformer:
                cat_feat_names_rain = list(onehot_encoder_rain.get_feature_names_out(original_cat_cols_rain_transformer))
            else:
                cat_feat_names_rain = []
                
            all_feat_names_rain = list(num_feat_names_rain) + cat_feat_names_rain
            
            importances_rain = regressor_fitted_rain.feature_importances_

            if len(all_feat_names_rain) == len(importances_rain):
                feature_importance_rain_df = pd.DataFrame({'feature': all_feat_names_rain, 'importance': importances_rain})
                feature_importance_rain_df = feature_importance_rain_df.sort_values('importance', ascending=False).reset_index(drop=True)

                N = 15
                plt.figure(figsize=(10, max(5, N*0.35)))
                sns.barplot(x='importance', y='feature', data=feature_importance_rain_df.head(N), palette='crest')
                plt.title(f'Top {N} Características - Predicción Rainfall (RFR)', fontsize=16)
                plt.xlabel('Importancia', fontsize=12)
                plt.ylabel('Característica', fontsize=12)
                plt.tight_layout()
                plt.show()
            else:
                print(f"Error en feat. imp. (Rainfall): Nombres ({len(all_feat_names_rain)}) vs Importancias ({len(importances_rain)}).")
        except Exception as e:
            print(f"Error al generar importancia de características para RFR (Rainfall): {e}")
else:
    print("Los datos de entrenamiento/preprocesador para regresión de Rainfall no están disponibles. Modelado omitido.")

### Celda 20: Modelo de Regresión para `Rainfall` (RandomForestRegressor con Log-Transform y `GridSearchCV`)

**Propósito:** Entrenar, optimizar y evaluar un modelo `RandomForestRegressor` para predecir la cantidad de lluvia (`Rainfall`). Dado que `Rainfall` es una variable con asimetría positiva (muchos ceros y algunos valores altos), se aplica una transformación logarítmica a la variable objetivo.

**Detalles del Código:**
1.  **Inicialización:** `r2_rain`, `rmse_rain`, `best_rf_rain_model`, y `y_pred_reg_rain` se inicializan.
2.  **Verificación:** Se comprueba que los datos de entrenamiento (`X_train_reg_rain`, `y_train_reg_rain_log`), el conjunto de prueba `y_test_reg_rain` (para evaluación final en escala original), y el preprocesador (`preprocessor_reg_rain`) estén disponibles.
3.  **Transformación Logarítmica:** `y_train_reg_rain_log` (que se calculó en la celda de preparación de datos de regresión usando `np.log1p`) se utiliza para entrenar el modelo.
4.  **Pipeline y GridSearchCV:**
    *   Se crea una `Pipeline` con `preprocessor_reg_rain` y `RandomForestRegressor`.
    *   Se reutiliza `param_grid_rf_reg` y `cv_kfold` (definidos previamente para el regresor de `MaxTemp`) o se definen si no existen.
    *   `GridSearchCV` se ajusta usando `y_train_reg_rain_log`.
5.  **Predicción e Inversión de Transformación:**
    *   Las predicciones (`y_pred_reg_rain_log`) se hacen en la escala logarítmica.
    *   `np.expm1` se usa para invertir la transformación y obtener `y_pred_reg_rain` en la escala original (mm).
    *   Se asegura que las predicciones de lluvia no sean negativas usando `np.maximum(0, ...)`.
6.  **Evaluación:** R² y RMSE se calculan comparando `y_test_reg_rain` (valores reales en escala original) con `y_pred_reg_rain`.
7.  **Importancia de Características:** Similar al modelo de `MaxTemp`, se visualizan las características más importantes para predecir `Rainfall`.

**Resultado Esperado:**
Si los datos están listos:
*   Mejores hiperparámetros y R² de validación cruzada (en escala logarítmica) para el `RandomForestRegressor` de `Rainfall`.
*   Métricas R² y RMSE en el conjunto de prueba, evaluadas en la escala original de `Rainfall`.
*   Un gráfico de barras mostrando las características más importantes para predecir `Rainfall`.
Si los datos no están listos, un mensaje indicando que se omitió el modelado.

In [ ]:
# --- Modelado con XGBoost (si está disponible) ---

# Inicializar variables para evitar errores si XGBoost no se ejecuta o falla
y_pred_xgb_temp, y_pred_xgb_rain = None, None
best_xgb_temp_model, best_xgb_rain_model = None, None

if xgb_available:
    print("✅ XGBoost está disponible. Entrenando modelos XGBoost...\n")

    # --- XGBoost para MaxTemp ---
    if 'X_train_reg_temp' in globals() and X_train_reg_temp is not None and \
       'y_train_reg_temp' in globals() and y_train_reg_temp is not None and \
       'preprocessor_reg_temp' in globals() and preprocessor_reg_temp is not None:
        
        print("--- Entrenando XGBRegressor para MaxTemp ---")
        pipeline_xgb_temp = Pipeline(steps=[
            ('preprocessor', preprocessor_reg_temp),
            ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42, 
                                       n_estimators=100, learning_rate=0.1,
                                       # Parámetros para evitar warnings comunes con versiones recientes
                                       # tree_method='hist', enable_categorical=True # si se usan cats directamente en XGB
                                      ))
        ])
        
        # Hiperparámetros para XGBRegressor (un conjunto pequeño para demostración)
        param_grid_xgb = {
            'regressor__n_estimators': [100, 150], # Reducido para demostración
            'regressor__learning_rate': [0.05, 0.1, 0.2],
            'regressor__max_depth': [3, 5, 7]
        }
        if 'cv_kfold' not in globals(): # Definir si no existe globalmente
            cv_kfold = KFold(n_splits=5, shuffle=True, random_state=42)

        grid_search_xgb_temp = GridSearchCV(pipeline_xgb_temp, param_grid_xgb, cv=cv_kfold, scoring='r2', n_jobs=-1, verbose=0)
        grid_search_xgb_temp.fit(X_train_reg_temp, y_train_reg_temp)
        
        print(f"Mejores hiperparámetros XGB (MaxTemp): {grid_search_xgb_temp.best_params_}")
        best_xgb_temp_model = grid_search_xgb_temp.best_estimator_
        y_pred_xgb_temp = best_xgb_temp_model.predict(X_test_reg_temp)

        print("\nXGBoost – Predicción de MaxTemp:")
        print(f"  R² (CV): {grid_search_xgb_temp.best_score_:.3f}")
        print(f"  R² (Test): {r2_score(y_test_reg_temp, y_pred_xgb_temp):.3f}")
        print(f"  RMSE (Test): {mean_squared_error(y_test_reg_temp, y_pred_xgb_temp, squared=False):.2f}°C")
    else:
        print("Datos para XGBoost (MaxTemp) no disponibles. Modelado omitido.")

    # --- XGBoost para Rainfall (con Log-Transform) ---
    if 'X_train_reg_rain' in globals() and X_train_reg_rain is not None and \
       'y_train_reg_rain_log' in globals() and y_train_reg_rain_log is not None and \
       'y_test_reg_rain' in globals() and y_test_reg_rain is not None and \
       'preprocessor_reg_rain' in globals() and preprocessor_reg_rain is not None:
        
        print("\n--- Entrenando XGBRegressor para Rainfall (con Log-Transform) ---")
        pipeline_xgb_rain = Pipeline(steps=[
            ('preprocessor', preprocessor_reg_rain),
            ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42,
                                       n_estimators=100, learning_rate=0.1,
                                       # tree_method='hist', enable_categorical=True
                                      ))
        ])
        
        # Reutilizar param_grid_xgb (definido arriba)
        grid_search_xgb_rain = GridSearchCV(pipeline_xgb_rain, param_grid_xgb, cv=cv_kfold, scoring='r2', n_jobs=-1, verbose=0)
        grid_search_xgb_rain.fit(X_train_reg_rain, y_train_reg_rain_log) # Entrenar con y_log

        print(f"Mejores hiperparámetros XGB (Rainfall): {grid_search_xgb_rain.best_params_}")
        best_xgb_rain_model = grid_search_xgb_rain.best_estimator_
        y_pred_xgb_rain_log = best_xgb_rain_model.predict(X_test_reg_rain)
        
        y_pred_xgb_rain = np.expm1(y_pred_xgb_rain_log) # Invertir log-transform
        y_pred_xgb_rain = np.maximum(0, y_pred_xgb_rain) # Asegurar no negatividad

        print("\nXGBoost – Predicción de Rainfall (evaluado en escala original):")
        print(f"  R² (CV, escala log): {grid_search_xgb_rain.best_score_:.3f}")
        print(f"  R² (Test, escala original): {r2_score(y_test_reg_rain, y_pred_xgb_rain):.3f}")
        print(f"  RMSE (Test, escala original): {mean_squared_error(y_test_reg_rain, y_pred_xgb_rain, squared=False):.2f} mm")
    else:
        print("Datos para XGBoost (Rainfall) no disponibles. Modelado omitido.")
else:
    print("❌ XGBoost no está instalado en este entorno. Se omiten los modelos XGBoost.")

### Celda 21: Modelos con XGBoost

**Propósito:** Entrenar, optimizar y evaluar modelos `XGBRegressor` para las tareas de predicción de `MaxTemp` y `Rainfall`, si la librería XGBoost está instalada y disponible.

**Detalles del Código:**
1.  **Inicialización:** Variables para predicciones y modelos XGBoost (`y_pred_xgb_temp`, `best_xgb_temp_model`, etc.) se inicializan a `None`.
2.  **Verificación de Disponibilidad:** El código se ejecuta solo si `xgb_available` es `True`.
3.  **XGBoost para `MaxTemp`:**
    *   Se verifica la disponibilidad de los datos de entrenamiento y el preprocesador.
    *   Se crea una `Pipeline` con `preprocessor_reg_temp` y un `XGBRegressor`. Se incluyen parámetros comentados (`tree_method`, `enable_categorical`) que pueden ser útiles para versiones más recientes de XGBoost y para manejar características categóricas directamente si no se usa OneHotEncoding.
    *   Se define `param_grid_xgb` para los hiperparámetros de XGBoost.
    *   `GridSearchCV` con `cv_kfold` y `scoring='r2'` optimiza el modelo.
    *   Se evalúa y se imprimen R² (CV y Test) y RMSE (Test).
4.  **XGBoost para `Rainfall` (con Log-Transform):**
    *   Se verifica la disponibilidad de los datos relevantes (incluyendo `y_train_reg_rain_log`).
    *   Se crea una `Pipeline` similar, usando `preprocessor_reg_rain`.
    *   `GridSearchCV` se ajusta usando `y_train_reg_rain_log`.
    *   Las predicciones se invierten (`np.expm1`) y se ajustan para no ser negativas.
    *   Se evalúa y se imprimen métricas (R² de CV en escala log, R² de Test y RMSE de Test en escala original).
5.  **Mensaje Alternativo:** Si XGBoost no está instalado, se imprime un mensaje indicándolo.

**Resultado Esperado:**
Si XGBoost está instalado y los datos están listos:
*   Mensajes de progreso y los mejores hiperparámetros para cada modelo XGBoost.
*   Métricas R² y RMSE para la predicción de `MaxTemp` con XGBoost.
*   Métricas R² y RMSE para la predicción de `Rainfall` con XGBoost (evaluadas en la escala original).
Si XGBoost no está instalado o los datos no están listos, se mostrarán los mensajes correspondientes.

In [ ]:
# --- Visualización: Comparación Predicción vs. Valor Real ---

# Gráfico para MaxTemp
if 'y_test_reg_temp' in globals() and y_test_reg_temp is not None:
    plt.figure(figsize=(10, 7))
    plot_title_temp = "Predicción de MaxTemp vs. Real"
    predictions_exist_temp = False

    # RandomForest Predicciones para MaxTemp
    if 'y_pred_reg_temp' in globals() and y_pred_reg_temp is not None and not np.isnan(r2_temp):
        sns.scatterplot(x=y_test_reg_temp, y=y_pred_reg_temp, alpha=0.6, edgecolor=None, s=50, label="Predicciones RF")
        plot_title_temp += f"\nRF R²: {r2_temp:.3f}, RMSE: {rmse_temp:.2f}°C"
        predictions_exist_temp = True
    
    # XGBoost Predicciones para MaxTemp (si existen)
    if xgb_available and 'y_pred_xgb_temp' in globals() and y_pred_xgb_temp is not None:
        try:
            xgb_r2_temp_plot = r2_score(y_test_reg_temp, y_pred_xgb_temp)
            xgb_rmse_temp_plot = mean_squared_error(y_test_reg_temp, y_pred_xgb_temp, squared=False)
            if not np.isnan(xgb_r2_temp_plot):
                 sns.scatterplot(x=y_test_reg_temp, y=y_pred_xgb_temp, alpha=0.6, color='darkorange', marker='x', s=50, label="Predicciones XGB")
                 plot_title_temp += f"\nXGB R²: {xgb_r2_temp_plot:.3f}, RMSE: {xgb_rmse_temp_plot:.2f}°C"
                 predictions_exist_temp = True
        except Exception as e:
            print(f"Advertencia: No se pudieron calcular/plotear métricas XGB para MaxTemp: {e}")


    if predictions_exist_temp:
        min_val_temp = y_test_reg_temp.min() 
        max_val_temp = y_test_reg_temp.max()
        # Ajustar min/max si las predicciones están fuera del rango de y_test
        if y_pred_reg_temp is not None:
            min_val_temp = min(min_val_temp, pd.Series(y_pred_reg_temp).min())
            max_val_temp = max(max_val_temp, pd.Series(y_pred_reg_temp).max())
        if xgb_available and y_pred_xgb_temp is not None:
            min_val_temp = min(min_val_temp, pd.Series(y_pred_xgb_temp).min())
            max_val_temp = max(max_val_temp, pd.Series(y_pred_xgb_temp).max())

        plt.plot([min_val_temp, max_val_temp], [min_val_temp, max_val_temp], 'r--', lw=2, label="Predicción Perfecta")
        
        plt.xlabel("Temperatura Máxima Real (°C)", fontsize=12)
        plt.ylabel("Temperatura Máxima Predicha (°C)", fontsize=12)
        plt.title(plot_title_temp, fontsize=14)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.axis('equal') 
        plt.tight_layout()
        plt.show()
    else:
        print("No hay predicciones válidas disponibles para graficar MaxTemp.")
else:
    print("No se pueden generar gráficos de predicción vs real para MaxTemp (y_test_reg_temp no disponible).")


# Gráfico para Rainfall
if 'y_test_reg_rain' in globals() and y_test_reg_rain is not None:
    plt.figure(figsize=(10, 7))
    plot_title_rain = "Predicción de Rainfall vs. Real"
    predictions_exist_rain = False

    # RandomForest Predicciones para Rainfall
    if 'y_pred_reg_rain' in globals() and y_pred_reg_rain is not None and not np.isnan(r2_rain):
        sns.scatterplot(x=y_test_reg_rain, y=y_pred_reg_rain, alpha=0.5, edgecolor=None, s=40, label="Predicciones RF")
        plot_title_rain += f"\nRF R²: {r2_rain:.3f}, RMSE: {rmse_rain:.2f} mm"
        predictions_exist_rain = True

    # XGBoost Predicciones para Rainfall (si existen)
    if xgb_available and 'y_pred_xgb_rain' in globals() and y_pred_xgb_rain is not None:
        try:
            xgb_r2_rain_plot = r2_score(y_test_reg_rain, y_pred_xgb_rain)
            xgb_rmse_rain_plot = mean_squared_error(y_test_reg_rain, y_pred_xgb_rain, squared=False)
            if not np.isnan(xgb_r2_rain_plot):
                sns.scatterplot(x=y_test_reg_rain, y=y_pred_xgb_rain, alpha=0.5, color='darkorange', marker='x', s=40, label="Predicciones XGB")
                plot_title_rain += f"\nXGB R²: {xgb_r2_rain_plot:.3f}, RMSE: {xgb_rmse_rain_plot:.2f} mm"
                predictions_exist_rain = True
        except Exception as e:
            print(f"Advertencia: No se pudieron calcular/plotear métricas XGB para Rainfall: {e}")
        
    if predictions_exist_rain:
        min_val_rain = 0 
        # Calcular max_val_rain_data basado en y_test y las predicciones disponibles
        max_val_rain_data = y_test_reg_rain.max()
        if y_pred_reg_rain is not None:
             max_val_rain_data = max(max_val_rain_data, pd.Series(y_pred_reg_rain).max())
        if xgb_available and y_pred_xgb_rain is not None:
            max_val_rain_data = max(max_val_rain_data, pd.Series(y_pred_xgb_rain).max())
            
        plot_max_rain = min(max_val_rain_data, np.percentile(y_test_reg_rain[y_test_reg_rain > 0], 99) * 1.5 if (y_test_reg_rain > 0).any() else max_val_rain_data)
        plot_max_rain = max(plot_max_rain, 10) # Asegurar un rango mínimo visible si toda la lluvia es baja

        plt.plot([min_val_rain, plot_max_rain], [min_val_rain, plot_max_rain], 'r--', lw=2, label="Predicción Perfecta")
        
        plt.xlabel("Lluvia Real (mm)", fontsize=12)
        plt.ylabel("Lluvia Predicha (mm)", fontsize=12)
        plt.title(plot_title_rain, fontsize=14)
        
        plt.xlim(left=-1, right=plot_max_rain + 1) 
        plt.ylim(bottom=-1, top=plot_max_rain + 1)

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        # plt.axis('equal') # Puede ser problemático para Rainfall si los rangos son muy diferentes. Mantener los límites explícitos.
        plt.tight_layout()
        plt.show()
    else:
        print("No hay predicciones válidas disponibles para graficar Rainfall.")
else:
    print("No se pueden generar gráficos de predicción vs real para Rainfall (y_test_reg_rain no disponible).")

### Celda 22: Comparación Gráfica: Predicción vs. Valor Real (Temperatura y Lluvia)

**Propósito:** Visualizar qué tan bien se alinean las predicciones de los modelos de regresión (RandomForest y, opcionalmente, XGBoost) con los valores reales para `MaxTemp` y `Rainfall`.

**Detalles del Código:**
1.  **Gráfico para `MaxTemp`:**
    *   Verifica si `y_test_reg_temp` existe.
    *   Si las predicciones de RandomForest (`y_pred_reg_temp`) y sus métricas (`r2_temp`) son válidas, se plotean.
    *   Si XGBoost está disponible y sus predicciones (`y_pred_xgb_temp`) son válidas, se calculan sus métricas y se superponen en el gráfico con un marcador y color diferente.
    *   Se calcula un rango dinámico para la línea de "Predicción Perfecta" (y=x) basado en los valores reales y todas las predicciones disponibles.
    *   El título del gráfico se actualiza dinámicamente para incluir las métricas de los modelos ploteados.
    *   `plt.axis('equal')` asegura escalas iguales en los ejes para una correcta interpretación de la línea y=x.
2.  **Gráfico para `Rainfall`:**
    *   Proceso similar al de `MaxTemp`.
    *   Para `plot_max_rain` (límite superior del gráfico de lluvia), se usa el percentil 99 de los valores reales de lluvia mayores que cero para evitar que outliers extremos distorsionen la visualización. Si no hay lluvia > 0, se usa el máximo. Se asegura un valor mínimo para `plot_max_rain` para una mejor visualización si toda la lluvia es muy baja.
    *   Los límites de los ejes X e Y se establecen explícitamente. `plt.axis('equal')` se comenta para Rainfall, ya que a veces puede hacer que el gráfico sea menos legible si los rangos de predicción y real difieren mucho.
3.  **Manejo de Errores:** Se incluyen mensajes si los datos necesarios para los gráficos no están disponibles.

**Resultado Esperado:**
*   Dos gráficos de dispersión:
    *   Uno para `MaxTemp`, mostrando los valores reales vs. los predichos por RandomForest (y XGBoost si está disponible).
    *   Uno para `Rainfall`, con una estructura similar.
*   Ambos gráficos incluirán una línea diagonal roja de "Predicción Perfecta".
*   Los títulos de los gráficos mostrarán las métricas R² y RMSE para los modelos correspondientes.
*   Mensajes de advertencia o informativos si no se pueden generar los gráficos o calcular métricas.

In [ ]:
# --- Resumen de Métricas Clave ---
print("--- Resumen de Métricas Clave de los Modelos Optimizados ---")

# Clasificación
lr_auc_final, rf_cls_auc_final = np.nan, np.nan 

if 'best_lr_model' in globals() and best_lr_model is not None and \
   'y_test_cls' in globals() and y_test_cls is not None and \
   'y_proba_lr' in globals() and y_proba_lr is not None:
    try:
        lr_auc_final = roc_auc_score(y_test_cls, y_proba_lr)
        print(f"\nClasificación - Regresión Logística (RainTomorrow):")
        print(f"  ROC AUC (Test): {lr_auc_final:.4f}")
    except Exception as e:
        print(f"Error al obtener métricas finales LR Clasificación: {e}")

if 'best_rf_cls_model' in globals() and best_rf_cls_model is not None and \
   'y_test_cls' in globals() and y_test_cls is not None and \
   'y_proba_rf_cls' in globals() and y_proba_rf_cls is not None:
    try:
        rf_cls_auc_final = roc_auc_score(y_test_cls, y_proba_rf_cls)
        print(f"\nClasificación - Random Forest (RainTomorrow):")
        print(f"  ROC AUC (Test): {rf_cls_auc_final:.4f}")
    except Exception as e:
        print(f"Error al obtener métricas finales RF Clasificación: {e}")

# Regresión MaxTemp (usando r2_temp, rmse_temp de la celda de modelado de MaxTemp RFR)
if 'r2_temp' in globals() and not np.isnan(r2_temp) and \
   'rmse_temp' in globals() and not np.isnan(rmse_temp):
    print(f"\nRegresión - Random Forest (MaxTemp):")
    print(f"  R² (Test): {r2_temp:.3f}")
    print(f"  RMSE (Test): {rmse_temp:.2f}°C")
else:
    print("\nMétricas para Random Forest (MaxTemp) no disponibles.")


# Regresión Rainfall (usando r2_rain, rmse_rain de la celda de modelado de Rainfall RFR)
if 'r2_rain' in globals() and not np.isnan(r2_rain) and \
   'rmse_rain' in globals() and not np.isnan(rmse_rain):
    print(f"\nRegresión - Random Forest (Rainfall):")
    print(f"  R² (Test): {r2_rain:.3f}")
    print(f"  RMSE (Test): {rmse_rain:.2f} mm")
else:
    print("\nMétricas para Random Forest (Rainfall) no disponibles.")

# XGBoost (si se ejecutó y las predicciones están disponibles)
if xgb_available:
    print("\n--- Métricas XGBoost (si los modelos se entrenaron y evaluaron) ---")
    # XGBoost MaxTemp
    if 'y_pred_xgb_temp' in globals() and y_pred_xgb_temp is not None and \
       'y_test_reg_temp' in globals() and y_test_reg_temp is not None:
        try:
            xgb_r2_temp_final = r2_score(y_test_reg_temp, y_pred_xgb_temp)
            xgb_rmse_temp_final = mean_squared_error(y_test_reg_temp, y_pred_xgb_temp, squared=False)
            if not np.isnan(xgb_r2_temp_final):
                print(f"\nRegresión - XGBoost (MaxTemp):")
                print(f"  R² (Test): {xgb_r2_temp_final:.3f}")
                print(f"  RMSE (Test): {xgb_rmse_temp_final:.2f}°C")
            else:
                print("\nMétricas para XGBoost (MaxTemp) no calculadas (R² es NaN).")
        except Exception as e:
            print(f"Error al calcular métricas finales XGB Temp: {e}")
    else:
        print("\nPredicciones XGBoost (MaxTemp) o datos de prueba no disponibles para resumen.")

    # XGBoost Rainfall
    if 'y_pred_xgb_rain' in globals() and y_pred_xgb_rain is not None and \
       'y_test_reg_rain' in globals() and y_test_reg_rain is not None:
        try:
            xgb_r2_rain_final = r2_score(y_test_reg_rain, y_pred_xgb_rain)
            xgb_rmse_rain_final = mean_squared_error(y_test_reg_rain, y_pred_xgb_rain, squared=False)
            if not np.isnan(xgb_r2_rain_final):
                print(f"\nRegresión - XGBoost (Rainfall):")
                print(f"  R² (Test): {xgb_r2_rain_final:.3f}")
                print(f"  RMSE (Test): {xgb_rmse_rain_final:.2f} mm")
            else:
                print("\nMétricas para XGBoost (Rainfall) no calculadas (R² es NaN).")
        except Exception as e:
            print(f"Error al calcular métricas finales XGB Rain: {e}")
    else:
        print("\nPredicciones XGBoost (Rainfall) o datos de prueba no disponibles para resumen.")

### Celda 23: Resumen de Métricas, Conclusiones y Próximos Pasos

**Propósito:** Esta sección finaliza el análisis resumiendo los hallazgos clave, discutiendo las limitaciones y sugiriendo posibles direcciones para trabajos futuros. La celda de código asociada imprime un resumen de las métricas de rendimiento de los modelos clave.

**Detalles del Código (Celda de Python asociada):**
*   El código Python imprime un resumen de las métricas de rendimiento finales (ROC AUC para clasificación; R² y RMSE para regresión) obtenidas en los conjuntos de prueba para los modelos optimizados.
*   Se verifica la existencia de los modelos y las predicciones necesarias antes de intentar calcular o imprimir las métricas, para evitar errores si alguna parte del modelado fue omitida o falló.
*   Las métricas de RandomForest para regresión (`r2_temp`, `rmse_temp`, `r2_rain`, `rmse_rain`) se toman de las variables globales calculadas en sus respectivas celdas de modelado.
*   Las métricas de XGBoost se recalculan aquí si sus predicciones están disponibles, para asegurar que se usan los datos correctos del conjunto de prueba.
*   Se incluyen bloques `try-except` y verificaciones de `np.isnan` para un resumen más robusto.

---

**Conclusiones Generales del Análisis Climático y Modelado:**

1.  **Análisis Exploratorio de Datos (EDA) y Geopandas:**
    *   El análisis se centró en tres ubicaciones alrededor de Melbourne: Melbourne, MelbourneAirport y Watsonia, mostrando patrones climáticos consistentes con su geografía.
    *   Se identificaron patrones estacionales claros en la temperatura, lluvia y humedad, con visualizaciones como lineplots, boxplots y heatmaps que ayudaron a comprender estas tendencias.
    *   La integración de `geopandas` y `folium` permitió la visualización espacial de las estaciones y datos agregados, mientras que `plotly.express` facilitó la exploración interactiva de relaciones multivariadas a lo largo del tiempo.

2.  **Modelado de Clasificación (`RainTomorrow`):**
    *   Se entrenaron modelos de Regresión Logística y Random Forest para predecir la probabilidad de lluvia al día siguiente.
    *   El desbalance de clases en `RainTomorrow` fue abordado mediante `class_weight='balanced'` y validación cruzada estratificada, mejorando la sensibilidad de los modelos a la clase minoritaria (lluvia).
    *   La optimización de hiperparámetros con `GridSearchCV` fue crucial para maximizar el rendimiento, medido principalmente por ROC AUC. Random Forest generalmente superó a la Regresión Logística.
    *   Características como la humedad (especialmente `Humidity3pm`), la presión atmosférica y la ocurrencia de lluvia el día actual (`RainToday`) demostraron ser predictores importantes.

3.  **Modelado de Regresión (`MaxTemp` y `Rainfall`):**
    *   Se desarrollaron modelos `RandomForestRegressor` (y opcionalmente `XGBRegressor`) para predecir valores continuos de temperatura máxima y cantidad de lluvia.
    *   **Predicción de `MaxTemp`:** Los modelos lograron un rendimiento notablemente bueno, con altos valores de R² y bajos RMSE, indicando que la temperatura máxima es relativamente predecible dadas las condiciones meteorológicas del día.
    *   **Predicción de `Rainfall`:** La predicción de la cantidad de lluvia resultó más desafiante, lo cual es común dada su naturaleza más estocástica. La transformación logarítmica de la variable `Rainfall` fue un paso importante para normalizar su distribución y mejorar el rendimiento del modelo, aunque los valores de R² fueron modestos.

**Limitaciones del Estudio:**

*   **Alcance Geográfico:** El análisis se limitó a solo tres ubicaciones; una cobertura más amplia podría revelar patrones climáticos más diversos y complejos.
*   **Imputación de Nulos:** Se utilizó imputación simple (media/moda). Estrategias más sofisticadas podrían preservar mejor la varianza y las relaciones en los datos.
*   **Ingeniería de Características:** Aunque se crearon características basadas en la fecha, una ingeniería más profunda (ej., variables de lag, interacciones complejas, índices climáticos) podría desbloquear un mayor poder predictivo.
*   **Validación Temporal:** El `train_test_split` estándar con mezcla (implícito en `GridSearchCV` con `KFold(shuffle=True)` para regresión) no respeta estrictamente la naturaleza secuencial de los datos climáticos. Esto podría llevar a una sobreestimación del rendimiento del modelo en un escenario de pronóstico real.
*   **Complejidad del Modelo vs. Interpretabilidad:** Aunque modelos como Random Forest y XGBoost son potentes, su interpretabilidad directa puede ser menor que modelos más simples.

**Próximos Pasos y Mejoras Potenciales:**

1.  **Ingeniería de Características Avanzada:** Incorporar variables de lag (ej., lluvia o temperatura de días anteriores), términos de interacción significativos, y promedios móviles para capturar mejor las dinámicas temporales.
2.  **Modelos Específicos para Series Temporales:** Explorar algoritmos como ARIMA, SARIMA, Prophet, o modelos de aprendizaje profundo como LSTMs, que están diseñados para datos secuenciales.
3.  **Validación Cruzada Temporal Robusta:** Implementar `TimeSeriesSplit` de scikit-learn o estrategias de validación forward-chaining para una evaluación más realista del rendimiento predictivo.
4.  **Análisis Geoespacial Extendido:** Con datos de más estaciones, realizar interpolaciones espaciales (ej., kriging) para crear mapas continuos, o analizar la autocorrelación espacial para entender la influencia de ubicaciones cercanas.
5.  **Manejo Avanzado de Desbalance de Clases:** Para la clasificación, explorar técnicas de remuestreo como SMOTE (Synthetic Minority Over-sampling Technique) o ADASYN, además del ajuste de pesos.
6.  **Optimización de Hiperparámetros Más Exhaustiva:** Utilizar técnicas de búsqueda más avanzadas como la optimización Bayesiana, o rangos de parámetros más amplios si los recursos computacionales lo permiten.
7.  **Análisis de Incertidumbre:** Cuantificar la incertidumbre en las predicciones, especialmente para variables críticas como la cantidad de lluvia.

Este proyecto mejorado establece una base sólida para el análisis climático y la aplicación de machine learning, destacando la importancia de un preprocesamiento cuidadoso, una evaluación rigurosa del modelo y la interpretabilidad de los resultados.